In [57]:
import plotly.graph_objects as go
import numpy as np
import hyperspy.api as hs
from plotly.subplots import make_subplots
# from scipy.signal import find_peaks

from utils.get_raw_data import  locate_raw_data, get_multiple_data_arrays
from utils.get_raw_data import get_array_and_names

emsa = locate_raw_data()

In [86]:
# from my notes
i_beam = [0.2, 0.2, 0.4, 0.1, 1.6, 1.6, 0.8, 0.2, 1.6, 1.6, 0.4, 0.2, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 0.2, 0.2]
dt = [10, 30, 15, 22, 33, 65, 58, 30, 30, 66, 42, 35, 5, 18, 18, 12, 27, 65, 25, 40]

In [87]:
s = get_multiple_data_arrays(every=True, normalize=False)

In [88]:
def print_lines(file, lines):
    with open(file, 'r') as f:
        for i, line in enumerate(f):
            if i in lines:
                lst = float(line.split(':')[1][1:-1])
    return lst

def get_live_time():
    lst = []
    l = [15] 
    for f in emsa:
        lst.append(print_lines(f, l))
    return lst

live_time = get_live_time()
live_time

[120.0,
 58.0,
 120.0,
 120.0,
 44.0,
 55.0,
 46.429515839,
 120.0,
 120.0,
 68.0,
 67.0,
 44.0,
 120.0,
 120.0,
 120.0,
 120.0,
 104.0,
 45.0,
 120.0,
 86.834312439]

In [89]:
sums = []
for i in range(len(s)):
    sums.append(s[i][1].sum())
maxs = []
for i in range(len(s)):
    maxs.append(s[i][1].max())
names = [n.split('_')[0] for n, _ in s]
v = [n.split('_')[1][:-2] for n, _ in s]
# make 05 into 5 in v
for i in range(len(v)):
    if v[i][0] == '0':
        v[i] = v[i][1:]
v

['5',
 '10',
 '30',
 '30',
 '5',
 '10',
 '15',
 '30',
 '5',
 '10',
 '15',
 '30',
 '5',
 '10',
 '15',
 '30',
 '5',
 '10',
 '15',
 '30']

In [94]:
# make latex table

perc = '\%'
d = '&'

header = f'{"Sample":20} {d} {"V$_acc$ [kV]":15} {d} {"I$_beam$ [nA]":15} {d} Dead time [\\{"%]":5}' 
header += f' {d} {"Live time [s]":15} {d} {"Highest peak [k counts]":15} {d} {"Total [k counts]":15} \\\\'
header += '\n\\hline'
print(header)

for i in range(len(names)):
    print(f'{names[i]:20} {d} {v[i]:15} {d} {i_beam[i]:15} {d} {dt[i]:5} {d} {live_time[i]:15.0f} {d} {maxs[i]/1000:15.0f} {d} {sums[i]/1000:15.0f} \\\\')
print('\n\n\n\n\n\n\n\n')

Sample               & V$_acc$ [kV]    & I$_beam$ [nA]   & Dead time [\%]    & Live time [s]   & Highest peak [k counts] & Total [k counts] \\
\hline
Cu                   & 5               &             0.2 &    10 &             120 &              18 &             155 \\
Cu                   & 10              &             0.2 &    30 &              58 &               5 &              62 \\
Cu                   & 30              &             0.4 &    15 &             120 &              94 &            1060 \\
Fe                   & 30              &             0.1 &    22 &             120 &             197 &            1864 \\
GaAs                 & 5               &             1.6 &    33 &              44 &              49 &            1172 \\
GaAs                 & 10              &             1.6 &    65 &              55 &             164 &            3715 \\
GaAs                 & 15              &             0.8 &    58 &              46 &             112 &            2653

In [91]:
# put into a plotly table

fig = go.Figure(data=[go.Table(
    header=dict(values=['Sample', 'V$_acc$ [kV]', 'I$_beam$ [nA]', 'Dead time [\\%]', 'Live time [s]', 'Highest peak [k counts]', 'Total [k counts]']),
    cells=dict(values=[names, v, i_beam, dt, live_time, [m/1000 for m in maxs], [s/1000 for s in sums]]))
])

fig.show()